# Labor 8 Grovers Suche mit einer unbekannten Anzahl von Lösungen

Voraussetzung

- [Kapitel 3.8 Grovers Algorithmus](https://qiskit.org/textbook/ch-algorithms/grover.html)
- [Kap.3.9 Quantenzählung](https://qiskit.org/textbook/ch-algorithms/quantum-counting.html)

Andere relevante Materialien

- [Abschnitt 3.3 Messen von T1 in Kap.6.1](https://learn.qiskit.org/course/quantum-hardware-pulses/calibrating-qubits-using-qiskit-pulse#T1)
- [QCQI] Michael A. Nielsen und Isaac L. Chuang. 2011. Quantencomputation und Quanteninformation

In [1]:
from qiskit import *
from qiskit.tools.visualization import plot_histogram
from qiskit.quantum_info import Operator, Statevector
from qiskit.tools.monitor import job_monitor
from qiskit.ignis.mitigation.measurement import *

import numpy as np
import matplotlib.pyplot as plt

<h2 style="font-size:24px;">Teil 1: Quantenzählung</h2>

<br>
<div style="background: #E8E7EB; border-radius: 5px;
-moz-border-radius: 5px;">
  <p style="background: #800080;
            border-radius: 5px 5px 0px 0px;
            padding: 10px 0px 10px 10px;
            font-size:18px;
            color:white;
            "><b>Tor</b></p>
    <p style=" padding: 0px 0px 10px 10px;
              font-size:16px;">Konstruieren Sie eine Schaltung zur Quantenzählung, die den IPE-Algorithmus (Iterative Phase Estimation) implementiert, um die Anzahl der Lösungen für ein Suchproblem zu finden.</p>
</div>

In [Kapitel 3.10 Grovers Algorithmus](https://qiskit.org/textbook/ch-algorithms/grover.html) haben wir in [Kapitel 3.11](https://qiskit.org/textbook/ch-algorithms/quantum-counting.html) Quantenzählung gelernt, wie man Suchproblemlösungen durch Grovers Algorithmus findet, und die Anzahl der Lösungen, die die Quantenzählschaltung verwenden. Die Anzahl der Lösungen zusammen mit der Anzahl der gesamten Elemente im Suchraum bestimmt die Anzahl der Grover-Iterationen und die Anzahl der erforderlichen Orakelaufrufe. In Teil 1 dieser Übung bauen wir die Quantenzählschaltung, die den IPE-Algorithmus implementiert, und nicht so, wie die Schaltung in [Kapitel 3.11 Quantenzählung](https://qiskit.org/textbook/ch-algorithms/quantum-counting.html) mit Quantenphasenschätzung (QPE) erstellt wurde.

<h3 style="font-size: 20px">1. Finden Sie die Anzahl der Lösungen des gegebenen Orakels für ein Suchproblem durch Quantenzählung.</h3> 

<h4 style="font-size: 17px">Schritt A. Konstruieren Sie ein Gatter für die Grover-Iteration.</h4>

Betrachten Sie den Suchraum mit der Gesamtzahl der Elemente, $N = 8$. Führen Sie die folgende Zelle aus, um ein Orakel eines Suchproblems zu erstellen.

In [2]:
## Create an Oracle

N = 8 # the number of total items in the search space
m = int(np.log2(N)) # the number of qubits required to construct the search space with N items

myqc = QuantumCircuit(m, name='Oracle')
myqc.x(1)
myqc.z(range(2))
myqc.x(1)

Oracle = myqc.to_gate()

📓 Vervollständigen Sie die Schaltung, `qc` , um das Grover-Iterationsgatter/den Operator `Grover` zu erstellen, indem Sie den Diffusor hinzufügen, der als Schritt 3 im ersten Abschnitt 1. `1.Introdcution` von [Kapitel 3.10 Grovers Algorithmus erklärt wird](https://qiskit.org/textbook/ch-algorithms/grover.html) .

In [3]:
qc = QuantumCircuit(m)
qc.append(Oracle, range(m))

### your code goes here













####

Grover = qc.to_gate()

<h4 style="font-size: 17px">📓Schritt B. Erstellen Sie eine Quantenschaltung, <code>circ</code> , für die Quantenzählung, indem Sie den IPE-Algorithmus verwenden, um den Eigenwert des Grover-Iterators <code>Grover</code> zu finden, den wir in Schritt A erstellt haben.</h4>

Lesen Sie [Kapitel 3.11 Quantenzählung](https://qiskit.org/textbook/ch-algorithms/quantum-counting.html) , bevor Sie beginnen. Angenommen, die Anzahl der Iterationen des IPE ist hier drei, was drei zählenden Qubits in der QPE-Schaltung (Quantum Phase Estimate) entspricht. (Mit anderen Worten, stellen Sie die Nummer des klassischen Registers drei ein.) 

In [1]:
###### your code goes here




























###################    
circ.draw()

<h4 style="font-size: 17px">📓Schritt C. Führen Sie die Schaltung aus, die Sie in Schritt B gebaut haben, und finden Sie die Anzahl der Lösungen, $M$, aus der geschätzten Phase.</h4>

In [3]:
sim = Aer.get_backend('qasm_simulator')
shots = 20000

In [2]:
####### Your code goes here
















<h2 style="font-size:24px;">Teil 2: Implementierung von Grovers Algorithmus mit einem erweiterten Orakel</h2>

<br>
<div style="background: #E8E7EB; border-radius: 5px;
-moz-border-radius: 5px;">
  <p style="background: #800080;
            border-radius: 5px 5px 0px 0px;
            padding: 10px 0px 10px 10px;
            font-size:18px;
            color:white;
            "><b>Tor</b></p>
    <p style=" padding: 0px 0px 10px 10px;
              font-size:16px;">Konstruieren Sie ein neues erweitertes Orakel, um den Suchraum zu verdoppeln, wenn die Anzahl der Lösungen $M$ größer oder gleich der Anzahl der Gesamtelemente $N$ ist, $M \geq N/2$.</p>
</div>

Wenn die Anzahl der Lösungen $M$ größer oder gleich der Hälfte der gesamten Items $N$ ( $M \geq N/2$ ) ist, wird der Winkel $\theta (= \arcsin(2\sqrt {M(NM)}/N) )$, der Rotationsbetrag hin zu den Lösungen durch jede Grover-Iteration, wird kleiner, wenn $M$ von $N/2$ bis $N$ variiert. Daher steigt die Anzahl der vom Suchalgorithmus benötigten Orakelaufrufe eher mit $M$, obwohl es einfacher sein sollte, eine Lösung für das Problem zu finden, wenn die Mehrheit der Elemente eine Lösung ist. In Teil 2 dieses Labs erstellen wir ein neues erweitertes Orakel, das den Suchbereich verdoppelt, um das Problem zu lösen. 

<h3 style="font-size: 20px">1. Verstehen Sie das Problem.</h3>

<h4 style="font-size: 17px">📓Schritt A. Überprüfen Sie, ob der Winkel $\theta$ kleiner wird, wenn $M$ von $N/2$ bis $N$ variiert.</h4>

Zeichnen Sie die Beziehung zwischen $M$ und $\theta$, wenn $N = 2^{10}$.

In [8]:
## Your code goes here

















<h4 style="font-size: 17px">📓Schritt B. Ermitteln Sie den Winkel $\theta$ und die Anzahl der Grover-Iterationen, $R$, die benötigt werden, um die Lösungen des Orakels in Teil 1 zu finden, und interpretieren Sie das Ergebnis.</h4>

In [QCQI] p253 wird $R$ durch $R = CI(\frac{\arccos \sqrt{M/N}}{\theta})$ geschätzt, wobei $\theta$ aus $\sin\theta bestimmt wird = \frac{2\sqrt{M(NM)}}{N}$ und $CI(x)$ bezeichnet die ganze Zahl, die der reellen Zahl $x$ am nächsten kommt, wobei wir gemäß Konvention die Hälfte abrunden, $CI(3.5) =3$, zum Beispiel.

In [3]:
N = 8
## Your code goes here














<h3 style="font-size: 20px">2. Finden Sie die Lösungen des Suchproblems aus Teil 1.</h3>

Die Lösungen können immer noch durch Grovers Algorithmus gefunden werden, wenn $M \geq N/2$ durch Verdoppeln des Suchraums mit einem einzelnen zusätzlichen Qubit $|q\rangle$ im Suchindex und Erstellen eines neuen erweiterten Orakels mit der Gesamtzahl der Elemente , $2N$ und $M$ Anzahl von Lösungen.

<h4 style="font-size: 17px">📓Schritt A. Erstellen Sie ein neues erweitertes Orakeltor/einen neuen Operator, <code>Oracle_new</code> , im verdoppelten Suchraum.</h4>

Mit diesem neuen Orakel im doppelten Suchraum ist das Problem nun definiert, $M$ Lösungen aus 16 ( = 2x$N$ ) Gesamtelementen zu finden. Daher sind jetzt weniger als die Hälfte der Elemente im neuen Suchraum Lösungen.

Wie in [QCQI] Kap.6.1.4 (p255) erklärt, markiert das neue erweiterte Orakel ein Element nur dann, wenn es eine Lösung für das Suchproblem ist und das zusätzliche Bit auf Null gesetzt ist. Das erweiterte Orakel kann unter Verwendung einer Anwendung des ursprünglichen Orakels, `Oracle` in Teil 1, und elementaren Quantengattern unter Verwendung des zusätzlichen Qubits $|q\rangle$ konstruiert werden.

In [6]:
## your code goes here




















<h4 style="font-size: 17px">📓Schritt B. Bewerten Sie die Anzahl der Grover-Iterationen, $R$, die benötigt werden, um $M$-Lösungen unter den insgesamt 16 Elementen zu finden.</h4>

In [5]:
## Your code goes here







<h4 style="font-size: 17px">📓Schritt C. Erstellen Sie eine Quantenschaltung <code>qc_final</code> , um Lösungen für das Suchproblem zu finden, indem Sie die Grover-Iteration <code>R</code> -mal anwenden.</h4>

Ein `diffuser` -Gate, das aus einer Grover-Iteration mit `Oracle_new` besteht, sollte entsprechend für den neuen Suchraum gebaut werden. Überprüfen Sie Abschnitt 3.3.1 `Qiskit Implementation` in [Kapitel 3.10 Grovers Algorithmus](https://qiskit.org/textbook/ch-algorithms/grover.html) , um zu erfahren, wie ein allgemeiner Diffusor erstellt wird.

In [7]:
## Your code goes here
























##########################

qc_final.draw()

In [8]:
count = execute(qc_final, sim ,shots=shots).result().get_counts()
plot_histogram(count)

<h4 style="font-size: 17px">📓Schritt D. Überprüfen Sie, ob die Lösungen korrekt sind, indem Sie das ursprüngliche Orakel, <code>Oracle</code> , in Teil 1 verwenden.</h4>

In [9]:
## your code goes here





















<h2 style="font-size:24px;">Teil 3: Grover-Schaltung auf Noisy Quantum System</h2>

<br>
<div style="background: #E8E7EB; border-radius: 5px;
-moz-border-radius: 5px;">
  <p style="background: #800080;
            border-radius: 5px 5px 0px 0px;
            padding: 10px 0px 10px 10px;
            font-size:18px;
            color:white;
            "><b>Tor</b></p>
    <p style=" padding: 0px 0px 10px 10px;
              font-size:16px;">Führen Sie die Grover-Schaltung, die wir in Teil 2 gebaut haben, auf einem realen Quantensystem aus.</p>
</div>

In Teil 3 führen wir die Schaltung `qc_final` , die wir in Teil 2 aufgebaut haben, um die Lösungen des Suchproblems auf einem realen Quantensystem zu finden. Da die heutigen Quantencomputer verrauscht sind, wird die Antwort daraus nicht mit dem Simulationsergebnis übereinstimmen, das wir in Teil 2 erhalten haben. Wir untersuchen, wie Rauschen das Ergebnis beeinflusst, und diskutieren die möglichen Fehlerquellen.

<h4 style="font-size: 17px">Schritt A. Führen Sie die folgende Zelle aus, um Ihr Konto zu laden und das Backend festzulegen.</h4>

In [16]:
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_athens')

<h4 style="font-size: 17px">📓Schritt B. Generieren Sie mehrere (beliebig viele) transpilierte Schaltungen von <code>qc_final</code> . Wählen Sie einen mit der minimalen Schaltungstiefe.</h4>

Wie wir in Lab5 gelernt haben, können wir die Genauigkeit der Ergebnisse von verrauschten Quantensystemen erhöhen, indem wir die Tiefe der von uns betriebenen Schaltung minimieren. 

In [17]:
## your code goes here












<h4 style="font-size: 17px">📓Schritt C. Führen Sie die Schaltung im Backend aus.</h4>

In [21]:
shots = 8192
## Your code goes here







<h4 style="font-size: 17px">📓Schritt D. Zeichnen Sie das Histogramm des Ergebnisses von <code>ibmq_athens</code> zusammen mit dem Simulationsergebnis, um zu vergleichen und zu diskutieren, wie Rauschen das Ergebnis beeinflusst.</h4>

Denken Sie darüber nach, warum einige der Lösungen stärker von Rauschen betroffen sind als andere.

In [10]:
## Your code goes here





